## 우편번호 데이터 aggregation 연습하기
* data/zip.json 파일 사용

#### data/zip.json 파일 읽기

In [6]:
import json

with open('data/zip.json', 'r', encoding='utf8') as file:
    contents = file.read()
    json_data = json.loads(contents)

In [7]:
print(type(json_data)) # 읽어온 데이터를 출력해보기
json_data[:3]

<class 'list'>


[{'_id': '01001',
  'city': 'AGAWAM',
  'loc': [-72.622739, 42.070206],
  'pop': 15338,
  'state': 'MA'},
 {'_id': '01002',
  'city': 'CUSHMAN',
  'loc': [-72.51565, 42.377017],
  'pop': 36963,
  'state': 'MA'},
 {'_id': '01005',
  'city': 'BARRE',
  'loc': [-72.108354, 42.409698],
  'pop': 4546,
  'state': 'MA'}]

---
#### MongoDB연결, DB생성, collection생성

In [11]:
import pymongo

# mongodb에 연결
conn = pymongo.MongoClient()
print(conn)

# db 생성
zip_db = conn.zip_db
print(zip_db)

# collection 생성
zip_col = zip_db.zip_col
print(zip_col)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'zip_db')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'zip_db'), 'zip_col')


---
#### zip_col에 zip.json 파일 insert 하기

In [12]:
zip_col.insert_many(json_data)

In [14]:
for z in zip_col.find()[:3]:
    print(z)

{'_id': '01001', 'city': 'AGAWAM', 'loc': [-72.622739, 42.070206], 'pop': 15338, 'state': 'MA'}
{'_id': '01002', 'city': 'CUSHMAN', 'loc': [-72.51565, 42.377017], 'pop': 36963, 'state': 'MA'}
{'_id': '01005', 'city': 'BARRE', 'loc': [-72.108354, 42.409698], 'pop': 4546, 'state': 'MA'}


---
#### 연습문제

In [16]:
# match와 field에 따라 collection을 조회하는 함수를 만든다
def find_col(m, f):
    for z in zip_col.find(m,f):
        print(z)
# match에 따라 aggregate하는 함수를 만든다
def agg_col(m):
    for z in zip_col.aggregate(m):
        print(z)

In [20]:
#state 별 인구수의 합계, _id는 출력하지 않고, 출력되는 documet를 10개로 제한한다.
f = [
    {
        "$group": {
            "_id":"$state",
            "totalPop":{ "$sum": "$pop"}
        }
    },
    {"$project": {"_id": 0, "totalPop": 1}},
    {"$limit": 10}
]
agg_col(f)

{'totalPop': 1003218}
{'totalPop': 4891769}
{'totalPop': 2350725}
{'totalPop': 4781379}
{'totalPop': 544698}
{'totalPop': 1108229}
{'totalPop': 4040587}
{'totalPop': 1578139}
{'totalPop': 606900}
{'totalPop': 798948}


In [23]:
#1. SQL: SELECT COUNT(*) AS count FROM zip
f = [
    {
        "$group": {
            "_id": None,
            "count":{"$sum":1}
        }
    }
]
agg_col(f)

{'_id': None, 'count': 29353}


In [24]:
#2. SQL: SELECT SUM(pop) as total_pop FROM zip
f = [
    {
        "$group": {
            "_id":None,
            "total_pop":{"$sum": "$pop"}
        }
    }
]
agg_col(f)

{'_id': None, 'total_pop': 248408400}


In [26]:
#3. SQL: SELECT state, SUM(pop) as total_pop FROM zip GROUP BY state
f = [
    {
        "$group":{
            "_id":"$state",
            "total_pop":{"$sum": "$pop"}
        }
    },
    {"$limit" : 10}
]
agg_col(f)

{'_id': 'AL', 'total_pop': 4040587}
{'_id': 'AR', 'total_pop': 2350725}
{'_id': 'HI', 'total_pop': 1108229}
{'_id': 'WA', 'total_pop': 4866692}
{'_id': 'MD', 'total_pop': 4781379}
{'_id': 'AK', 'total_pop': 544698}
{'_id': 'WI', 'total_pop': 4891769}
{'_id': 'RI', 'total_pop': 1003218}
{'_id': 'MT', 'total_pop': 798948}
{'_id': 'UT', 'total_pop': 1722850}


In [27]:
#4. SQL : select city, sum(pop) as total_pop from zip group by city
f = [
    {
        "$group":{
            "_id":"$city",
            "total_pop":{"$sum": "$pop"}
        }
    },
    {"$limit" : 10}
]
agg_col(f)

{'_id': 'GANSEVOORT', 'total_pop': 14485}
{'_id': 'QUINBY', 'total_pop': 0}
{'_id': 'ALLEGANY', 'total_pop': 7885}
{'_id': 'RICHTON', 'total_pop': 7378}
{'_id': 'JOELTON', 'total_pop': 6749}
{'_id': 'SPRINGBROOK', 'total_pop': 692}
{'_id': 'EDGAR', 'total_pop': 795}
{'_id': 'METHUEN', 'total_pop': 39664}
{'_id': 'WEST COVINA', 'total_pop': 94294}
{'_id': 'WATER VALLEY', 'total_pop': 6561}


In [37]:
#5. SQL: SELECT state, SUM(pop) as total_pop FROM zip GROUP BY state
# ORDER BY as total_pop
f = [
    {
        "$group":{
            "_id":"$state",
            "total_pop":{"$sum": "$pop"}
        }
    },
    {"$limit" : 10},
    {"$sort":{"total_pop": 1}}
]
agg_col(f)

{'_id': 'AK', 'total_pop': 544698}
{'_id': 'DC', 'total_pop': 606900}
{'_id': 'MT', 'total_pop': 798948}
{'_id': 'RI', 'total_pop': 1003218}
{'_id': 'HI', 'total_pop': 1108229}
{'_id': 'NE', 'total_pop': 1578139}
{'_id': 'AR', 'total_pop': 2350725}
{'_id': 'AL', 'total_pop': 4040587}
{'_id': 'MD', 'total_pop': 4781379}
{'_id': 'WI', 'total_pop': 4891769}


In [43]:
#6. # SQL: SELECT COUNT(*) FROM zip WHERE state = 'MA' 
f = [
    {
        "$match":{"state":"MA"}
    },
    {
        "$group":{
            "_id":None,
            "total_pop":{"$sum": 1}
        }
    },
]
agg_col(f)

{'_id': None, 'total_pop': 474}


In [42]:
#7. select state,sum(pop) as total_pop from zip 
# where state = 'MA' group by state
f = [
    {
        "$match":{"state":"MA"}
    },
    {
        "$group":{
            "_id":"$state",
            "total_pop":{"$sum": "$pop"}
        }
    },
]
agg_col(f)

{'_id': 'MA', 'total_pop': 6016425}


In [44]:
#7.1 select state,sum(pop) as total_pop from zip where state in ('DE', 'MS') 
# group by state
f = [
    {
        "$match":{"state":{"$in": ["DE", "MS"]}}
    },
    {
        "$group":{
            "_id":"$state",
            "total_pop":{"$sum": "$pop"}
        }
    },
]
agg_col(f)

{'_id': 'MS', 'total_pop': 2573216}
{'_id': 'DE', 'total_pop': 666168}


In [45]:
#8. SELECT state, SUM(pop) as total_pop FROM zip 
# GROUP BY state HAVING SUM(pop) > 10000000
f = [
    {
        "$group":{
            "_id":"$state",
            "total_pop":{"$sum": "$pop"}
        }
    },
    {
        "$match":{"total_pop":{"$gt": 10000000}}
    }
]
agg_col(f)

{'_id': 'FL', 'total_pop': 12686644}
{'_id': 'TX', 'total_pop': 16984601}
{'_id': 'CA', 'total_pop': 29754890}
{'_id': 'NY', 'total_pop': 17990402}
{'_id': 'PA', 'total_pop': 11881643}
{'_id': 'OH', 'total_pop': 10846517}
{'_id': 'IL', 'total_pop': 11427576}


In [52]:
#9.1000만 이상의 state 별 총 인구를 state_pop 필드명으로 출력하고 _id는 출력하지 않기
f = [
    {
        "$group":{
            "_id":"$state",
            "state_pop":{"$sum": "$pop"}
        }
    },
    {"$project": {"_id": 0}},
    {"$limit" : 10}
]
agg_col(f)

{'state_pop': 6478216}
{'state_pop': 4040587}
{'state_pop': 2350725}
{'state_pop': 1108229}
{'state_pop': 4866692}
{'state_pop': 4781379}
{'state_pop': 1003218}
{'state_pop': 4891769}
{'state_pop': 544698}
{'state_pop': 798948}


In [55]:
#10.1000만 이상의 state만 내림차순 정렬하여 3개만 가져오기
f = [
    {
        "$group":{
            "_id":"$state",
            "state_pop":{"$sum": "$pop"}
        }
    },
    {"$match": {"state_pop": {"$gt": 10000000}}},
    {"$sort":{"state_pop":-1}},
    {"$limit" : 3}
]
agg_col(f)

{'_id': 'CA', 'state_pop': 29754890}
{'_id': 'NY', 'state_pop': 17990402}
{'_id': 'TX', 'state_pop': 16984601}


In [56]:
#11.1000만 이상의 state 별 총 인구를 state_pop 필드명으로 출력하고,
# _id는 출력하지 않으며, 가장 많은 인구를 가진 3개만 출력하기
f = [
    {
        "$group":{
            "_id":"$state",
            "state_pop":{"$sum": "$pop"}
        }
    },
    {"$match": {"state_pop": {"$gt": 10000000}}},
    {"$sort":{"state_pop":-1}},
    {"$project": {"_id":0}},
    {"$limit" : 3}
]
agg_col(f)

{'state_pop': 29754890}
{'state_pop': 17990402}
{'state_pop': 16984601}


In [58]:
#12. select state, city, sum(pop) as total_pop from zip group by state,city 
f = [
    {
        "$group":{
            "_id":{
                "state":"$state",
                "city":"$city"
            },
            "state_pop":{"$sum": "$pop"}
        }
    },
    {"$limit" : 10}
]
agg_col(f)

{'_id': {'state': 'TN', 'city': 'PEGRAM'}, 'state_pop': 2742}
{'_id': {'state': 'NY', 'city': 'WARRENSBURG'}, 'state_pop': 4399}
{'_id': {'state': 'MD', 'city': 'HOLLYWOOD'}, 'state_pop': 6785}
{'_id': {'state': 'WV', 'city': 'GRANTSVILLE'}, 'state_pop': 1873}
{'_id': {'state': 'CT', 'city': 'TRUMBULL'}, 'state_pop': 32213}
{'_id': {'state': 'PA', 'city': 'MOUNT LEBANON'}, 'state_pop': 18660}
{'_id': {'state': 'MA', 'city': 'WHITMAN'}, 'state_pop': 13208}
{'_id': {'state': 'WI', 'city': 'SAINT CROIX FALL'}, 'state_pop': 1640}
{'_id': {'state': 'RI', 'city': 'TIVERTON'}, 'state_pop': 14310}
{'_id': {'state': 'PA', 'city': 'LANDENBERG'}, 'state_pop': 4787}


In [93]:
#13. select state, city, sum(pop) as total_pop from zip 
# GROUP BY state, city HAVING city = 'POINT BAKER'
f = [
    {
        "$group":{
            "_id":{
                "state":"$state",
                "city":"$city"
            },
            "total_pop":{"$sum": "$pop"}
        }
    },
    {"$match":{"_id.city": "POINT BAKER"}},
]
agg_col(f)

{'_id': {'state': 'AK', 'city': 'POINT BAKER'}, 'total_pop': 426}


In [78]:
#14. SELECT AVG(pop) FROM zip GROUP BY state, city  
f = [
    {
        "$group":{
            "_id":{
                "state":"$state",
                "city":"$city"
            },
            "avg_pop":{"$avg": "$pop"}
        }
    },
    {"$limit":10}
]
agg_col(f)

{'_id': {'state': 'NH', 'city': 'MUNSONVILLE'}, 'avg_pop': 535.0}
{'_id': {'state': 'PA', 'city': 'FRANKLIN'}, 'avg_pop': 18357.0}
{'_id': {'state': 'PA', 'city': 'MONT ALTO'}, 'avg_pop': 1601.0}
{'_id': {'state': 'MA', 'city': 'FT DEVENS'}, 'avg_pop': 8480.0}
{'_id': {'state': 'MO', 'city': 'SOUTH COUNTY'}, 'avg_pop': 45920.0}
{'_id': {'state': 'IA', 'city': 'BADGER'}, 'avg_pop': 1270.0}
{'_id': {'state': 'VA', 'city': 'ELKTON'}, 'avg_pop': 10331.0}
{'_id': {'state': 'TX', 'city': 'ART'}, 'avg_pop': 24.0}
{'_id': {'state': 'ME', 'city': 'NOBLEBORO'}, 'avg_pop': 1455.0}
{'_id': {'state': 'WI', 'city': 'REESEVILLE'}, 'avg_pop': 2027.0}


In [96]:
#15. select state,city, avg(pop) as avg_pop from zip  
# GROUP BY state, city  having avg_pop > 30000
#주별 도시 인구 평균이 30000 이 넘는 곳의 _id는 출력하지 않으며, 
# state 와 city 이름만 출력하기 (3개만 출력하기)
f = [
    {
        "$group":{
            "_id":{
                "city":"$city",
                "state":"$state"
            },
            "avg_pop":{"$avg": "$pop"}
        }
    },
    {"$match": {"avg_pop":{"$gt":30000}}},
    {"$limit":3},
    {"$project": {"avg_pop":0}},
#     {"$unwind":"$_id"},
]
agg_col(f)

{'_id': {'city': 'MOUNT HOLLY', 'state': 'SC'}}
{'_id': {'city': 'CAMPBELL', 'state': 'CA'}}
{'_id': {'city': 'HOBOKEN', 'state': 'NJ'}}
